<a href="https://colab.research.google.com/github/CyberTycoon/transformer-text-summarizer/blob/main/summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
import re

def preprocess_text(text):
    # Remove special characters but keep key punctuation
    text = re.sub(r'[^\w\s.,;!?]', '', text)
    # Ensure single spaces
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

article = """
         Bitcoin crossed $87,000 for the first time in weeks, even as concerns over US tariffs persist.

The cryptocurrency is up over 3% in the past 24 hours, trading at $87,430 — its highest since April 2.

It has gained 16% from its 2025 low of just under $75,000 on April 9, narrowing the gap from its record high to 20%.

Ethereum and XRP also extended their recovery on Monday.

The top 10 cryptocurrencies by market capitalization were mostly stable or slightly higher over the past day.

The global crypto market cap rose 3.24% to $2.76 trillion, while total market volume jumped 47.75% to $67.18 billion.

As markets stabilize, some investors are seeking early-stage opportunities with high-return potential.

One name drawing interest is PepeX, a meme coin launchpad built with an AI-powered infrastructure.

Crypto market’s comeback
Bitcoin, XRP, and other cryptocurrencies gained early Monday, supported by a weaker dollar.

Bitcoin surged late Sunday, holding above $87,000, as the dollar index slid to a three-year low following President Trump’s remarks about potentially removing Federal Reserve chair Jerome Powell.

National Economic Council Director Kevin Hassett had said on Friday that the administration is still reviewing the possibility of removing Federal Reserve Chair Jerome Powell.

Hassett further implied the Fed, under Powell’s leadership, had acted with political motives favoring Democrats.

The crypto rebound came alongside a fresh record in gold, reflecting rising investor demand for perceived safe-haven and inflation-hedging assets.

Why PepeX is drawing the market’s attention
PepeX is gaining momentum by presenting itself as a more structured, security-focused alternative in the meme coin launchpad sector.

As platforms like Pump.fun come under scrutiny for hosting low-quality, high-risk projects, PepeX offers a curated, AI-driven framework prioritizing transparency and developer accountability.

Its core tool, the Moonshot Engine, streamlines token creation through automation, while integrated promotional bots amplify project visibility across social media.

The platform also enforces a 5% cap on token holdings for project creators and a $500 launch fee, measures aimed at deterring opportunistic launches and signaling credibility.

With meme coin volumes still substantial and speculative interest holding firm, PepeX is positioning itself to meet demand for a safer, more reliable launchpad option.

PepeX presale: price to go up soon
PepeX’s presale has raised over $1.4 million as investor interest grows for a more structured, security-focused alternative to the often chaotic meme coin launchpad space.

The project’s token allocation prioritizes utility and sustainability, with 45% set aside for the presale, 10% each for development and liquidity, 15% for marketing, another 15% for staking and rewards, and 5% for treasury use.

The token is currently priced at $$0.0243, but it will soon jump up to $0.0255.

Early investors are set for massive gains once the presale ends, and may see more gains once the token is listed on exchanges.

PepeX is positioning itself with a pragmatic, security-first framework aimed at tackling issues like rug pulls, copycat projects, and lack of transparency — challenges that have long plagued the meme coin segment.



"""
clean_article = preprocess_text(article)

In [ ]:
def summarize(text,
              max_length=500,
              min_length=200,
              num_beams=4,
              temperature=1.0):
    inputs = tokenizer([text], max_length=1024, return_tensors="pt", truncation=True)

    summary_ids = model.generate(
        inputs["input_ids"],
        num_beams=num_beams,
        max_length=max_length,
        min_length=min_length,
        temperature=temperature,
        early_stopping=True
    )

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Example usage
summary = summarize(clean_article)
print(f"Summary: {summary}")

Summary: Crypto markets comeback Bitcoin, XRP, and other cryptocurrencies gained early Monday. Bitcoin crossed 87,000 for the first time in weeks, even as concerns over US tariffs persist. The top 10 cryptocurrencies by market capitalization were mostly stable or slightly higher over the past day. The global crypto market cap rose 3.24 to 2.76 trillion, while total market volume jumped 47.75 to 67.18 billion. As markets stabilize, some investors are seeking earlystage opportunities with highreturn potential. One name drawing interest is PepeX, a meme coin launchpad built with an AIpowered infrastructure. PepeXs presale has raised over 1.4 million as investor interest grows for a more structured, securityfocused alternative to the often chaotic meme coinLaunchpad space. The projects token allocation prioritizes utility and sustainability, with 45 set aside for the presale, 10 each for development and liquidity, 15 for marketing, another 15 for staking and rewards, and 5 for treasury use

In [ ]:
!pip install gradio
import gradio as gr

with gr.Blocks() as demo:
    with gr.Row():
        article = gr.Textbox(label="Paste Article", lines=10)
        summary = gr.Textbox(label="Generated Summary", lines=10)

    with gr.Row():
        length = gr.Slider(30, 300, value=150, label="Summary Length")
        generate = gr.Button("Generate Summary")
        feedback = gr.Radio(["👍 Good", "👎 Bad"], label="Feedback")

    generate.click(
        fn=summarize,
        inputs=[article, length],
        outputs=summary
    )

    feedback.change(
        fn=lambda fb: print(f"Feedback logged: {fb}"),
        inputs=feedback,
        outputs=None
    )

demo.launch()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 105.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.9 MB/s eta 0:00:00
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8e1458399f00fd5c65.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory